In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [2]:
#Load the data using pandas read_csv function. This will generate a data frame we will use through the work.
cleaned_goalkeeeprs_df = pd.read_csv('goalkeepers_rating.csv')
cleaned_midfielders_df = pd.read_csv('midfielders_rating.csv')
cleaned_forwarders_df = pd.read_csv('forwarders_rating.csv')
cleaned_defenders_df = pd.read_csv('defenders_rating.csv')

In [3]:
#Show the first and last few records to check data is loading correctly
print(cleaned_forwarders_df.head())
print(cleaned_forwarders_df.tail())

         date pos_role  original_rating  goals  assists  shots_ontarget  \
0  2016-06-10       FW             7.34      1        0               2   
1  2016-06-10      FWL             9.59      1        1               1   
2  2016-06-10       FW             5.84      0        0               0   
3  2016-06-10      FWR             6.46      0        0               0   
4  2016-06-11       FW             5.79      0        0               0   

   shots_offtarget  shotsblocked  chances2score  drib_success  ...  \
0                2             0              3             0  ...   
1                1             1              0             3  ...   
2                2             0              2             1  ...   
3                3             2              3             0  ...   
4                1             1              0             0  ...   

   missed_penalties  owngoals  degree_centrality  betweenness_centrality  \
0                 0         0           1.428571    

In [4]:
import pandas as pd

df = cleaned_forwarders_df

# Iterate through each column and check for columns with a single unique value
single_value_columns = [column for column in df.columns if df[column].nunique() == 1]

# Print columns with a single unique value
print("Columns with a single unique value:")
for column in single_value_columns:
    print(column)

Columns with a single unique value:
goals_ag_otb
goals_ag_itb
saves_otb
saved_pen


In [5]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['goals_ag_otb',
'goals_ag_itb',
'saves_otb',
'saved_pen']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_forwarders_df = cleaned_forwarders_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_forwarders_df.head()

,date,pos_role,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,2016-06-10,FW,7.34,1,0,2,2,0,3,0,...,0,0,1.428571,0.316015,0.754464,0.315068,1.090213,0.050000,90,90
1,2016-06-10,FWL,9.59,1,1,1,1,1,0,3,...,0,0,1.428571,0.176156,0.754464,0.547945,1.465723,0.055556,90,90
2,2016-06-10,FW,5.84,0,0,0,2,0,2,1,...,0,0,0.928571,0.133565,0.670635,0.173913,0.867720,0.000000,61,90
3,2016-06-10,FWR,6.46,0,0,0,3,2,3,0,...,0,0,1.000000,0.073594,0.635338,0.232877,2.026114,0.000000,66,90
4,2016-06-11,FW,5.79,0,0,0,1,1,0,0,...,0,0,0.857143,0.080159,0.635338,0.096386,0.552186,0.000000,60,90


In [6]:
cleaned_forwarders_df.shape

(2599, 49)

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import time

In [8]:
cleaned_forwarders_df.head()

,date,pos_role,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,2016-06-10,FW,7.34,1,0,2,2,0,3,0,...,0,0,1.428571,0.316015,0.754464,0.315068,1.090213,0.050000,90,90
1,2016-06-10,FWL,9.59,1,1,1,1,1,0,3,...,0,0,1.428571,0.176156,0.754464,0.547945,1.465723,0.055556,90,90
2,2016-06-10,FW,5.84,0,0,0,2,0,2,1,...,0,0,0.928571,0.133565,0.670635,0.173913,0.867720,0.000000,61,90
3,2016-06-10,FWR,6.46,0,0,0,3,2,3,0,...,0,0,1.000000,0.073594,0.635338,0.232877,2.026114,0.000000,66,90
4,2016-06-11,FW,5.79,0,0,0,1,1,0,0,...,0,0,0.857143,0.080159,0.635338,0.096386,0.552186,0.000000,60,90


In [9]:
cleaned_forwarders_df.describe()

,original_rating,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
count,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,...,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000,2599.000000
mean,7.024983,0.321277,0.106195,0.819931,1.135052,0.417468,1.359369,0.825317,0.663332,0.355521,...,0.011928,0.000770,1.122548,0.159492,0.678086,0.265520,1.001324,0.025041,82.109273,90.265487
std,0.875417,0.586194,0.336793,0.992219,1.153299,0.678507,1.320524,1.207287,0.938529,0.668636,...,0.108581,0.027735,0.257377,0.076793,0.081570,0.098358,0.571165,0.043209,12.647086,2.810188
min,4.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.000000,0.000000,0.012500,0.000000,0.000000,19.000000,90.000000
25%,6.370000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.933333,0.102261,0.622222,0.197183,0.607404,0.000000,76.000000,90.000000
50%,6.860000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.133333,0.148156,0.685714,0.259740,0.949069,0.000000,90.000000,90.000000
75%,7.520000,1.000000,0.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,1.285714,0.206798,0.725926,0.327869,1.349787,0.047619,90.000000,90.000000
max,10.000000,4.000000,3.000000,7.000000,6.000000,4.000000,8.000000,8.000000,6.000000,6.000000,...,1.000000,1.000000,1.866667,0.466606,0.933333,0.750000,6.074042,0.333333,120.000000,120.000000


In [10]:
cleaned_forwarders_df.original_rating.value_counts()

original_rating
6.25    23
6.44    22
6.52    21
6.39    21
6.21    21
        ..
7.99     1
9.55     1
8.84     1
5.55     1
9.25     1
Name: count, Length: 386, dtype: int64

In [11]:
# lets move towards splitting the data.

# get the locations
y = cleaned_forwarders_df['original_rating']
X = cleaned_forwarders_df.drop(columns=['original_rating'], axis=1)

In [12]:
X.head()

,date,pos_role,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,2016-06-10,FW,1,0,2,2,0,3,0,0,...,0,0,1.428571,0.316015,0.754464,0.315068,1.090213,0.050000,90,90
1,2016-06-10,FWL,1,1,1,1,1,0,3,1,...,0,0,1.428571,0.176156,0.754464,0.547945,1.465723,0.055556,90,90
2,2016-06-10,FW,0,0,0,2,0,2,1,2,...,0,0,0.928571,0.133565,0.670635,0.173913,0.867720,0.000000,61,90
3,2016-06-10,FWR,0,0,0,3,2,3,0,0,...,0,0,1.000000,0.073594,0.635338,0.232877,2.026114,0.000000,66,90
4,2016-06-11,FW,0,0,0,1,1,0,0,1,...,0,0,0.857143,0.080159,0.635338,0.096386,0.552186,0.000000,60,90


In [13]:
y.head()

0    7.34
1    9.59
2    5.84
3    6.46
4    5.79
Name: original_rating, dtype: float64

In [14]:
# Create DataFrame
# Convert 'date' column to datetime with error handling
X['date'] = pd.to_datetime(X['date'], errors='coerce')

# Check for any NaT values
if X['date'].isna().any():
    print("Some dates couldn't be parsed. Handling them as NaT (Not a Time).")

# Extract year, month, and day
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['day'] = X['date'].dt.day

# Drop the original 'date' column
X = X.drop(columns=['date'])

# Display the DataFrame
print(X.head())

  pos_role  goals  assists  shots_ontarget  shots_offtarget  shotsblocked  \
0       FW      1        0               2                2             0   
1      FWL      1        1               1                1             1   
2       FW      0        0               0                2             0   
3      FWR      0        0               0                3             2   
4       FW      0        0               0                1             1   

   chances2score  drib_success  drib_unsuccess  keypasses  ...  \
0              3             0               0          0  ...   
1              0             3               1          4  ...   
2              2             1               2          0  ...   
3              3             0               0          0  ...   
4              0             0               1          0  ...   

   betweenness_centrality  closeness_centrality  flow_centrality  \
0                0.316015              0.754464         0.315068   
1   

In [15]:
X.columns

Index(['pos_role', 'goals', 'assists', 'shots_ontarget', 'shots_offtarget',
       'shotsblocked', 'chances2score', 'drib_success', 'drib_unsuccess',
       'keypasses', 'touches', 'passes_acc', 'passes_inacc', 'crosses_acc',
       'crosses_inacc', 'lballs_acc', 'lballs_inacc', 'grduels_w', 'grduels_l',
       'aerials_w', 'aerials_l', 'poss_lost', 'fouls', 'wasfouled',
       'clearances', 'stop_shots', 'interceptions', 'tackles', 'dribbled_past',
       'tballs_acc', 'tballs_inacc', 'ycards', 'rcards', 'dangmistakes',
       'countattack', 'offsides', 'saves_itb', 'missed_penalties', 'owngoals',
       'degree_centrality', 'betweenness_centrality', 'closeness_centrality',
       'flow_centrality', 'flow_success', 'betweenness2goals', 'minutesPlayed',
       'game_duration', 'year', 'month', 'day'],
      dtype='object')

In [16]:
X.dtypes

pos_role                   object
goals                       int64
assists                     int64
shots_ontarget              int64
shots_offtarget             int64
shotsblocked                int64
chances2score               int64
drib_success                int64
drib_unsuccess              int64
keypasses                   int64
touches                     int64
passes_acc                  int64
passes_inacc                int64
crosses_acc                 int64
crosses_inacc               int64
lballs_acc                  int64
lballs_inacc                int64
grduels_w                   int64
grduels_l                   int64
aerials_w                   int64
aerials_l                   int64
poss_lost                   int64
fouls                       int64
wasfouled                   int64
clearances                  int64
stop_shots                  int64
interceptions               int64
tackles                     int64
dribbled_past               int64
tballs_acc    

In [17]:
X.pos_role.value_counts()

pos_role
FW     1997
FWL     302
FWR     300
Name: count, dtype: int64

In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = MinMaxScaler()  
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)
X = preprocessor.fit_transform(X)
# Get the numerical column names after scaling
num_col_names = num_features

# Get the categorical column names after one hot encoding
cat_col_names = preprocessor.named_transformers_['OneHotEncoder'].get_feature_names_out(cat_features)

# Combine all column names
all_col_names = list(num_col_names) + list(cat_col_names)
X= pd.DataFrame(X, columns=all_col_names)
print(X.head())

   goals  assists  shots_ontarget  shots_offtarget  shotsblocked  \
0    1.0      0.0             0.0             0.25      0.000000   
1    0.0      1.0             0.0             0.25      0.333333   
2    1.0      0.0             0.0             0.00      0.000000   
3    0.0      0.0             1.0             0.00      0.000000   
4    1.0      0.0             0.0             0.00      0.000000   

   chances2score  drib_success  drib_unsuccess  keypasses  touches  ...  \
0       0.285714      0.333333            0.00      0.375    0.000  ...   
1       0.142857      0.166667            0.25      0.000    0.375  ...   
2       0.000000      0.333333            0.00      0.250    0.125  ...   
3       0.000000      0.500000            0.50      0.375    0.000  ...   
4       0.000000      0.166667            0.25      0.000    0.000  ...   

   flow_success  betweenness2goals  minutesPlayed  game_duration      year  \
0      0.677264           0.808355       0.410262       0.1794

In [19]:
X.shape

(2599, 52)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, split the train+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Display the sizes of the splits to ensure they are correct
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 1559
Validation set size: 520
Test set size: 520


In [21]:
X_train.head()

,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,...,flow_success,betweenness2goals,minutesPlayed,game_duration,year,month,day,pos_role_FW,pos_role_FWL,pos_role_FWR
1106,1.0,0.0,0.0,0.00,0.0,0.142857,0.000000,0.0,0.125,0.125,...,0.454723,0.736842,0.326808,0.038462,0.000000,0.702970,0.0,0.5,1.000000,0.266667
1184,1.0,0.0,0.0,0.00,0.0,0.285714,0.166667,0.0,0.375,0.000,...,0.641634,0.808355,0.309161,0.181818,0.000000,0.702970,0.0,0.5,1.000000,0.400000
1909,1.0,0.0,0.0,0.25,0.0,0.428571,0.000000,0.0,0.250,0.000,...,0.555849,0.777778,0.282413,0.153846,0.230769,0.594059,0.0,1.0,0.181818,0.533333
2385,1.0,0.0,0.0,0.25,0.0,0.285714,0.166667,0.0,0.250,0.125,...,0.592625,0.875000,0.455342,0.205882,0.111111,0.702970,0.0,1.0,0.363636,0.400000
1781,1.0,0.0,0.0,0.25,0.0,0.285714,0.000000,0.0,0.125,0.250,...,0.246783,0.700000,0.334589,0.260870,0.166667,0.435644,0.0,1.0,0.181818,0.066667


In [26]:
# Save each DataFrame as a CSV file
X.to_csv('forwarders_rating_X.csv', index=False)
y.to_csv('forwarders_rating_y.csv', index=False)


print("DataFrames have been saved as CSV files.")

DataFrames have been saved as CSV files.
